In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optional, Tuple

class MultiHeadAttention( nn.Module ):
    """多头注意力，qkv三者维度被均分为num_heads个部分，增强并行能力和表示能力，更鲁棒"""
    def __init__( self, config: XXXConfig):
        super().__init__()
        # config参数内化：头数、维度等
        self.num_heads = config.num_heads      # 头数，即qkv的维度被均分为多少部分
        self.hidden_dim = config.hidden_dim    # 嵌入维度embedding_dim，即输入向量的最后一个维度
        self.qk_dim = config.qk_dim            # query和key投影矩阵的维度，两者需要点积因此维度必须一致，可以任意，但通常简化为与hidden_dim一致。
        self.v_dim = config.v_dim              # value投影矩阵的维度，可以与qk和hidden_dim不一致，但通常简化为与hidden_dim一致，如Baichuan2-7B就是三者都等于hidden_dim
        self.head_dim = self.hidden_dim // self.num_heads       # 也有直接设置为config.kv_channels指定的，如chatglm3-6b
        assert self.head_dim * self.num_heads == hidden_dim , "Embedding size must be divisible by num_heads"

        # 投影矩阵组件：下面三个投影矩阵可以写为一个self.W_pack，要用时再拆分
        self.query_linear = nn.Linear( self.hidden_dim, self.qk_dim )
        self.key_linear = nn.Linear( self.hidden_dim, self.qk_dim )
        self.value_linear = nn.Linear( self.hidden_dim, self.v_dim )
        self.out_linear = nn.Linear( self.v_dim, self.hidden_dim)

        # 旋转位置编码组件
        self.max_position_embeddings = config.max_position_embeddings
        self.rotary_emb = RotaryEmbedding(self.head_dim, max_position_embeddings=self.max_position_embeddings)
    
    def forward(
        self, 
        hidden_states,                                          # 输入的embedding结果
        attention_mask: Optional[torch.Tensor] = None,          # 掩码，用于训练和batch推理mask padding
        position_ids: Optional[torch.LongTensor] = None,        # 位置id，用于Rotary旋转位置编码组件
        past_key_value: Optional[Tuple[torch.Tensor]]  = None,  # 是否有之前的Kv_cache，区分首次迭代和之后
        use_cache: bool = False,                                # 是否启用kv_cache
    ):
        """
        inputs.shape = [batch_size, token_len, hidden_state]
        """
        batch_size, q_len = hidden_states.shape[0], hidden_states.shape[1]

        Q = self.query_linear( hidden_states )
        K = self.key_linear( hidden_states )
        V = self.value_linear( hidden_states )
        # 先view重塑再transpose，可以使得张量在内存中数据的排列方式符合后续多头并行计算：
        # view 操作要求张量在内存中是连续的（contiguous），view 不会改变张量在内存中的实际存储顺序，它只是重新解释张量的形状
        # transpose 不会改变张量在内存中的实际存储顺序，但它会改变张量的步幅（stride），从而改变访问数据的方式。
        # 先将 query 重塑为 (batch_size, seq_len, num_heads, head_dim)确保 seq_len 和 head_dim 在内存中是连续的。再将 num_heads 和 seq_len 的维度交换，改变了维度顺序，但保留了每个头的 seq_len 和 head_dim 的连续性。
        # 如果直接使用 query.view(batch_size, num_heads, -1, head_dim)，虽然形状是对的，但数据在内存中的排列可能不符合多头注意力的计算需求，因为 seq_len 和 head_dim 可能不再是连续的。
        Q = Q.view( batch_size, q_len, self.num_heads, self.head_dim ).transpose( 1, 2) 
        K = K.view( batch_size, q_len, self.num_heads, self.head_dim ).transpose( 1, 2)
        V = V.view( batch_size, q_len, self.num_heads, self.head_dim ).transpose( 1, 2)

        # 对QK进行位置编码：要求是获得当前长度
        kv_seq_len = K.shape[-2]
        if past_key_value != None:
            kv_seq_len += past_key_value[0].shape[-2]
        cos, sin = self.rotary_emb( value_states, seq_len = kv_seq_len)
        Q, K = apply_rotary_pos_emb(Q, K, cos, sin, position_ids)

        # 再拼接kv_cache中的K和V
        if past_key_value != None:
            K, V = torch.cat( [past_key_value[0], K], dim = 2 ), torch.cat( [past_key_value[1], V], dim = 2 )       # 在q_len维度进行拼接
        # 更新kv_cache
        if use_cache:
            past_key_value = (K, V)

        # 进行缩放点积SDPA
        attn_output = F.scaled_dot_product_attention(Q, K, V, attn_mask = attention_mask)       # 或设置is_causal=True，也是默认单向注意力
        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape( batch_size, q_len, self.hidden_dim)

        # 最后进行混淆
        attn_output = self.out_linear( attn_output )
        return attn_output, past_key_value

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, List, Tuple

class MultiQueryAttention( nn.Mudule ):
    """
    MQA和GQA，其中前者是后者一个特例，即group数量为1。
    """
    def __init__( self, config: XXXConfig ):
        # config参数内化
        self.hidden_dim = config.hidden_dim         # embedding维度
        self.qk_dim = config.qk_dim
        self.v_dim = config.value_dim

        self.num_heads = config.num_heads           # query组数
        self.head_dim = self.qk_dim // self.num_heads

        self.num_groups = config.num_groups         # kv组数，为1时是MQA，>1时为GQA
        self.query_per_kv = self.num_heads // self.num_groups
        assert self.query_per_kv * self.num_groups == self.num_heads, "GQA组数必须可以整除Query头数"

        # 线性层实例化
        self.query_linear = nn.Linear( self.hidden_dim, self.qk_dim * self.num_heads )
        self.key_linear = nn.Linear( self.hidden_dim, self.qk_dim * self.num_groups )
        self.value_linear = nn.Linear( self.hidden_dim, self.v_dim * self.num_groups )
        self.out_linear = nn.Linear( self.v_dim * self.num_groups * , self.v_dim * self.hidden_dim )

        # 位置编码层
        self.rotary_emb = RotaryEmbedding( self.qk_dim, max_rotary_embeddings = self.max_rotary_embeddings)
    
    def forward(
        self, 
        hidden_states,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        use_cache: bool = False
    ):
        """
        hidden_states.shape = [batch_size, q_lens, hidden_dim]
        """
        batch_size, q_lens = hidden_states.shape[0], hidden_states.shape[1]
        Q = self.query_linear( hidden_states ).view( batch_size, q_lens, self.num_heads, self.qk_dim )
        K = self.key_linear( hidden_states ).view( batch_size, q_lens, self.num_groups, self.qk_dim )
        V = self.value_linear( hidden_states ).view( batch_size, q_lens, self.num_groups, self.v_dim )
        
        # 位置编码
        kv_seq_len = K.shape[1]
        if past_key_value:
            kv_seq_len += past_key_value[0].shape[1]
        cos, sin = self.rotary_emb( V, kv_seq_len )
        Q, K = apply_rotary_pos_emb(Q, K, cos, sin, position_ids)

        # KV_cache：在seq维度扩展
        if past_key_value:
            K, V = torch.cat( [past_key_value[0], K], dim = 1), torch.cat( [past_key_value[1], V], dim = 1)
        if use_cache:
            past_key_value = (K. V)
        
        # 扩展以适应MQA和GQA点积
        # 将KV的[batch_size, q_lens, num_groups, dim]四个维度的倒数第二个维度处插入一个维度，变成[batch_size, q_lens, 1, num_groups, dim]
        # 并在该维度复制 num_heads // self.num_groups 份后与num_group维度合并，使得KV与Q在第三个维度的维数一致，便于计算
        K = K.unsqueeze( -2 )
        K = K.expand( -1, -1, -1, self.num_heads // self.num_groups, -1)
        K = K.contiguous().view( k.shape[:2] + ( self.num_heads ,self.qk_dim) )
        V = V.unsqueeze( -2 )
        V = V.expand( -1, -1, -1, self.num_heads // self.num_groups, -1)
        V = V.contiguous().view( k.shape[:2] + ( self.num_heads ,self.qk_dim) )

        # 调整形状为 batch_size, num_heads, q_lens, dim进行并行计算SDPA
        Q, K, V = [ states.transpose( 1,2 ) for states in [Q, K, V]]
        attn_output = F.scaled_dot_product_attention( Q, K, V, is_causal = True )
        attn_output.transpose( 1, 2)
        return attn_output, past_key_value

In [ ]:
import torch
import torch.nn as nn

class RotaryEmbedding( nn.Module ):
    """旋转位置编码，不是作用于embedding，而是Q和K"""
    def __init__(self, qk_dim, max_positions = 151643, base = 10000 ):
        """
        param qk_dim: Query和Key的维度
        max_positions: 预存的最大token长度
        base: 旋转角度的基数，即sin和cos的周期
        """
        super().__init__()
        self.qk_dim = qk_dim
        self.base = base
        self.max_positions = max_positions
        # 获取表示维度的三角函数角频率向量列表，定长，即base^(2i/qk_dim)
        self.inv_freq = 1.0 / self.base ** (torch.arrange(0, self.qk_dim, 2.0) / self.self.qk_dim)
        # 预存最大编码长度范围内的编码值（sin和cos），但不是直接编码，需要转换。届时可以直接根据position_ids索引获取
        self.rope_cache = self.get_cos_sin_cache( self.inv_freq, self.max_positions )

    def get_cos_sin_cache( self, inv_freq, max_positions ):
        "预存最大长度的位置编码，shape = [max_positions, qk_dim // 2, 2]，前一半是cos，后一半是sin"
        dtype = inv_freq.dtype
        token_pos = torch.arrange( 0, max_positions )
        # 每个pos的theta*pos：将角频率向量与位置id向量外积，并double cat，获得shape = [max_positions, qk_dim]的张量
        idx_theta = torch.outer( token_pos, inv_freq )
        rope_cache = torch.stack( [ torch.cos(idx_theta), torch.sin(idx_theta)], dim = -1)
        return rope_cache
    
    def forward( self, x, seq_len = None ):
        "根据实际序列长度和数据类型获取cos和sin"
        if seq_len == None:
            seq_len = 1
        if seq_len > self.max_positions:
            self.rope_cache = self.get_cos_sin_cache( self.inv_freq, seq_len)
        
        return self.rope_cache[:seq_len].to( dtype = x.dtype )


def apply_rotary_pos_emb( X, rope_cache):
    """
    param X: [batch_size, num_heads, seq_len, qk_dim]
    param rope_cache: [seq_len, qk_dim//2, 2]，cos和sin的编码
    [X2i, X2i+1] * [[ cos2i, -sin2i ]
                    [ sin2i+1, cos2i+1 ]] = [RoPE_X_2i, RoPE_X_2i+1]
    """
    batch_size, num_heads, seq_len, qk_dim = X.shape
    # 将最后一个维度拆分为qk_dim // 2个两两一组的相邻维度
    xshaped = X.reshape( batch_size, num_heads, seq_len, qk_dim // 2, 2)
    rope_cache = rope_cache.reshape( 1, 1, seq_len, qk_dim // 2, 2 )
    # 旋转位置编码：X2i = cos*X2i + sin*X2i+1, X2i+1 = -sin*X2i + cos*X2i+1
    x_out2 = torch.stack(
        [
            xshaped[..., 0] * rope_cache[..., 0] + xshaped[..., 1] * rope_cache[..., 1],
            xshaped[..., 1] * rope_cache[..., 0] - xshaped[..., 0] * rope_cache[..., 1]
        ],
        dim = -1
    )
    # 从第三维度开始，将x_out2的最后一个维度合并到第二个维度，即[batch_size, num_heads, seq_len, qk_dim]
    x_out2 = x_out2.flatten(3)
    return x_out2.reshape( batch_size, num_heads, seq_len, qk_dim)


In [ ]:
import torch
import torch.nn as nn
class MistralRotaryEmbedding(nn.Module):
    def __init__(self, qk_dim, max_positions = 151643, base = 10000):
        super().__init__()
        
        


In [4]:
-float('inf')

-inf